In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.sparkContext.setLogLevel("ERROR")
spark._jvm.decoders.HRTDCDecoder.registerUDF(spark._jsparkSession)
spark._jvm.decoders.HRTDCUnpairedDecoder.registerUDF(spark._jsparkSession)

df = spark.read.parquet("/home/h487/data/parquet/run1007.parquet")
#df.show(10)

# Filter SRPPAC strip data and decode
df_src = df.filter("femType==7 and femId==615").select("data").withColumn("decoded",F.expr("decode_hrtdc_unpaired_segdata(data)"))
df_src = df_src.select("decoded.*").select("hbf.*","data").select("hbfNumber","data").filter("array_size(decoded.data)>0")
df_src = df_src.withColumn("ex",F.explode("data")).select("hbfNumber","ex.*").withColumnRenamed("tot","edge")
#df_src.filter("hbfNumber=10057285 and ch=41").show()
df_src.show(5)

# Filter SRPPAC anode data and decode
df_sra = df.filter("femType==5 and femId==614").select("data").withColumn("decoded",F.expr("decode_hrtdc_segdata(data)"))
df_sra = df_sra.select("decoded.*").select("hbf.*","data").select("hbfNumber","data").filter("array_size(decoded.data)>0")
df_sra = df_sra.withColumn("ex",F.explode("data")).select("hbfNumber","ex.*").filter("ch==4")
df_sra = df_sra.groupBy("hbfNumber").agg(F.collect_list("time").alias("time"),F.collect_list("tot").alias("tot")) # select fastest anode hit
df_sra = df_sra.withColumn("size", F.size("time"))
df_sra.show(5)

check = spark.read.parquet("/home/h487/data/parquet/run1007_srppac.parquet")
check.show(20)

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
26/01/15 01:53:44 WARN Utils: Your hostname, gpuana02, resolves to a loopback address: 127.0.1.1; using 192.168.1.216 instead (on interface enp4s0)
26/01/15 01:53:44 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
26/01/15 01:53:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/15 01:53:45 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
26/01/15 01:53:45 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
26/01/15 01:53:45 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
26/01/15 01:53:45 WARN Utils: Se

+---------+---+---------+----+
|hbfNumber| ch|     time|edge|
+---------+---+---------+----+
| 12261748| 42|427374760|   0|
| 12261748| 22|427375129|   0|
| 12261748| 23|427375495|   0|
| 12261748| 41|427375973|   0|
| 12261748| 43|427377582|   0|
+---------+---+---------+----+
only showing top 5 rows
+---------+--------------------+--------------------+----+
|hbfNumber|                time|                 tot|size|
+---------+--------------------+--------------------+----+
| 12261748|         [427411599]|             [83315]|   1|
| 12261749|[222935691, 22902...|[83336, 83305, 83...|   4|
| 12261750|[224389826, 44441...|[83342, 83321, 83...|   3|
| 12261751|[11665250, 231942...|[83378, 83371, 83...|   3|
| 12261753|[281113819, 45394...|      [83306, 83308]|   2|
+---------+--------------------+--------------------+----+
only showing top 5 rows
+---------+------------+--------------------+--------------------+------+------------------+------------------+------------------+------------